# reactive
> Reactive calculations for on-demand calculation (a.k.a. excel calc graph)

In [ ]:
#| default_exp reactive

In [ ]:
#| hide
import pandas as pd
import numpy as np
import sklearn.datasets as ds
import sklearn.model_selection as ms
import sklearn.ensemble as en
from sklearn import linear_model

# Ractive Values and inputs

We define basic abstractions for calculations

In [ ]:
#| export
from fastcore.all import *
from nbdev import show_doc

In [ ]:
#| hide
class TraceCalcTree():
    "Helper method to trace calculations"
    trace = False
    def __enter__(self): 
        trace = True
    def __exit__(self): 
        trace = False
    @staticmethod
    def on_calced(value): 
        if TraceCalcTree.trace: print(f'Calculating node -> {value}')

## RModel

Tracks timesteps of the calculation.

Reactive values can use either the global singleton or a dedicated instance

In [ ]:
#| export
class RModel(): 
    "Reactive model defines global calculation timer"
    ts: int # current time step of the calculation
    def __init__(self): self.ts = 0
    def step(self): 
        "Increases the timestep of the reactive model"
        self.ts += 1
    __repr__ = basic_repr('ts')
    

In [ ]:
show_doc(RModel.step)

---

[source](https://github.com/jstranik/mana-signals/blob/main/mana_signals/reactive.py#L27){target="_blank" style="float:right; font-size:smaller"}

### RModel.step

>      RModel.step ()

Increases the timestep of the reactive model

In [ ]:
#| export 
singleton_model = RModel() # Global singleton for model (used if not passed down to calculations)

We also have a global variable `singleton_model` to represent a global singleton. 

## Basic Primitives

In [ ]:
#| export
class RValue():
    """Basic reactive calculation primitive

    The object represents a cachable reactive value. 

    Method `calc`() on an object is called whenever a value of the object is needed. 
    The value is cached in an object. 
    The value is automatically invalidated whenever its inputs changes.
    """
    __repr__ = basic_repr('cached_value,ts_checked,ts_updated,model')
    def __init__(self, model:RModel=singleton_model): 
        self.cached_value, self.ts_checked, self.ts_updated = (None, 0, 0)
        self.deps, self.model = None, model
    def set_model(self,model:RModel): 
        """Sets model for the calculated value. 
        Setting model also sets automatically model for all dependent values
        """
        self.model = model
        for v in self.get_dependents(): v.set_model(model)

    def invalidate_if_outdated(self): 
        if self.ts_checked < self.model.ts: 
            dep_ts = [v.invalidate_if_outdated() for v in self.get_dependents()]
            max_dep_ts = max(dep_ts) if dep_ts else 0
            if self.ts_updated < max_dep_ts: 
                self.ts_updated = max_dep_ts
                self.cached_value = None
            self.ts_checked = self.model.ts
        return self.ts_updated

    def is_outdated(self): 
        """Returns true if the value changed due to update to dependent inputs"""
        self.invalidate_if_outdated()
        return self.cached_value is None
        
    def get_dependents(self): 
        """Returns all RValues that this calculation depends on"""
        if self.deps is None: 
            self.deps = [ v for (n,v) in self.__dict__.items() if isinstance(v,RValue)]
        return self.deps
        
    @property
    def value(self): 
        """Returns value of the RValue object. 
        If the value is outdated, the calc method is automatically called
        """
        if self.model.ts > self.ts_checked :
            self.invalidate_if_outdated()
        if self.cached_value is None: self.cached_value = self.calc()
        return self.cached_value
  
    def calc(self): 
        """Calculates the value. Must be overriden"""
        raise NotImplementedError('RValue.calc method must be overriden')



In [ ]:
show_doc(RValue.get_dependents)

---

[source](https://github.com/jstranik/mana-signals/blob/main/mana_signals/reactive.py#L67){target="_blank" style="float:right; font-size:smaller"}

### RValue.get_dependents

>      RValue.get_dependents ()

Returns all RValues that this calculation depends on

In [ ]:
show_doc(RValue.value)

---

[source](https://github.com/jstranik/mana-signals/blob/main/mana_signals/reactive.py#L74){target="_blank" style="float:right; font-size:smaller"}

### RValue.value

>      RValue.value ()

Returns value of the RValue object. 
If the value is outdated, the calc method is automatically called

In [ ]:
show_doc(RValue.is_outdated)

---

### RValue.is_outdated

>      RValue.is_outdated ()

Returns true if the value changed due to update to dependent inputs

In [ ]:
show_doc(RValue.calc)

---

[source](https://github.com/jstranik/mana-signals/blob/main/mana_signals/reactive.py#L83){target="_blank" style="float:right; font-size:smaller"}

### RValue.calc

>      RValue.calc ()

Calculates the value. Must be overriden

In [ ]:
#| export 
class RInput(RValue):
    """
    Represents input value to the calculation

    Input values are `RValue`s that can be set using `RInput.set_value`.
    """
    def __init__(self, init_value): super().__init__(); self.cached_value = init_value
    def set_value(self, value): 
        """
        Sets the value of the RValue object. 
        Any other reactive values that depend on this value are automatically invalidated
        """
        self.model.step()
        self.ts_checked = self.ts_updated = self.model.ts
        self.cached_value = value
    def calc(self): raise NotImplementedError("Input value not provided")


In [ ]:
show_doc(RInput.set_value)

---

[source](https://github.com/jstranik/mana-signals/blob/main/mana_signals/reactive.py#L97){target="_blank" style="float:right; font-size:smaller"}

### RInput.set_value

>      RInput.set_value (value)

Sets the value of the RValue object. 
Any other reactive values that depend on this value are automatically invalidated

## Examples

### Caching of calculation

Most basic usage of reactive values. Here we create a reactive value with 2 inputs and calculate the result

In [ ]:
a = RInput(1)
b = RInput(2)
class RCalc(RValue):
    def __init__(self,a,b): super().__init__(); self.a,self.b = a,b
    def calc(self):
        print("calculating value c")
        return self.a.value + self.b.value
c= RCalc(a,b)
c.value




calculating value c


3

In [ ]:
c.value


3

In [ ]:
a.set_value(5)

In [ ]:
c.value

calculating value c


7

### Simplified definition of calculation

Let's make writing calculated values a little bit easier.

In [ ]:
#| export
def rcalc(func):
    """
    A decorator for converting a simple function to a reactive value
    """
    class RCalcClass(RValue):
        def __init__(self, *args):
            super().__init__()
          
            self.deps = args
            
        def calc(self):
            return func(*[a.value for a in self.deps])

    RCalcClass.__name__ = func.__name__.capitalize()
    return RCalcClass

In [ ]:
a = RInput(1)
b = RInput(2)

@rcalc
def adder(a,b): return a+b
c = adder(a,b)
c.value

3

In [ ]:
a.set_value(7)
c.value


9

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()